# Running open AR-Sandbox without a sandbox

The software can also be run in fully virtual mode, without an AR-Sandbox attached to it - for example to test new implementations or to evaluate the software itself, before investing time or funds to obtain an AR-Sandbox.

For this purpose, the sensor can be set to a `dummy`-sensor and a DEM can be loaded (for example from a previously saved version, or one of the DEM's supplied as additional data sets - more info below).

We show here an example with the topography-module - the method can, of course, also be combined with other modules.

In [1]:
#Only use if sandbox is not installed
# import os,sys
# sys.path.append('../../../')

In [2]:
# %load_ext autoreload
# %autoreload 2

## Initialize Sensor and Projector

Sensor initialisation starts out as before:

In [3]:
#save the paths to the calibration files 
from sandbox import _calibration_dir
_calibprojector = _calibration_dir + "my_projector_calibration.json"
_calibsensor = _calibration_dir + "my_sensor_calibration.json"

## Setting a `dummy`-sensor

**Note**: use `dummy`-sensor to start without actual AR-Sandbox connection: define with

```name="dummy"```

in sensor call:

In [4]:
#Import Sensor
from sandbox.sensor import Sensor
# sensor = Sensor(calibsensor=_calibsensor, name="kinect_v2")
sensor = Sensor(_calibsensor, name="dummy")
# test

sandbox.sensor.sensor_api: INFO | JSON configuration loaded for sensor.
sandbox.sensor.dummy: INFO | DummySensor initialized.


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


The next cell will create the window for the "projector" - in the offline version, simply open it in a separate window to follow the next steps.

In [5]:
# Import projector
from sandbox.projector import Projector
projector = Projector(calibprojector=_calibprojector)


sandbox.projector.projector: INFO | JSON configuration loaded for projector


Launching server at http://localhost:53882
sandbox.projector.projector: INFO | Projector initialized and server started.
Please position the browser window accordingly and enter fullscreen!


## Initialize marker detection

We can also set ArUco marker positions directly by coordinates:

In [ ]:
#Initialize the aruco detection
from sandbox.markers import MarkerDetection
aruco = MarkerDetection(sensor=sensor)

In [ ]:
aruco.set_aruco_position(dict_position={1:[50,30],2:[100,80]}, frame=sensor.get_frame())
aruco.df_aruco_position
aruco.set_aruco_position(dict_position={}, frame=sensor.get_frame())



## Load test data


We now load topography data - here from one of the digital evlevation models supplied with open-AR-Sandbox (if the file is not found, then download the dataset with the `Download_datasets.ipynb` notebook):

In [1]:
import numpy as np

In [2]:
# file = np.load(test_data['topo'] + "DEM1.npz")
file = np.load("..\\06_LoadSaveTopoModule\saved_DEMs\DEM1.npz")
frame = file['arr_0'] # + np.min(file['arr_0'])
extent = np.asarray([0, frame.shape[1], 0, frame.shape[0], frame.min(), frame.max()])

Reset the frame position (shift to minimum value - not necessarily required, depending on the DEM):

In [3]:
frame -= np.min(frame)

## Initialize main thread for live update

Create main thread - also as in normal mode:

In [ ]:
from sandbox.main_thread import MainThread
main = MainThread(sensor=sensor, projector=projector, aruco=aruco if 'aruco' in globals() else None)


Now, assing loaded frame to `main`-thread:

In [ ]:
main.load_frame(frame)

In [ ]:
# Start the thread 
main.run()

After this step, the loaded DEM should also appear in the projector browser tab/ window.

### Control the features of the main thread

As in normal mode, we can now adjust settings in the projection, for example change color-map, line spacing, etc.:

In [ ]:
main.widget_plot_module()

For further use, we can also save the generated projector figure (for example, to generate teaching material, visualization in external software, etc.):

In [ ]:
# test saving figure
if True:
    projector.figure.savefig("terrain_plot.png")
    # projector.figure.axes[0].plot([50,100],[50,100], 'k-')
    # projector.figure.axes[0].contourf(frame)

## Import the desired module

In [ ]:
# Import the modules to use
from sandbox.modules import TopoModule


In [ ]:
TopoModule = TopoModule(extent=sensor.extent)

## Add the module to the main thread

In [ ]:
main.add_module(name ='Topo', module=TopoModule)

In [ ]:
TopoModule.sea_level_polygon_alpha = 0.7

In [ ]:
main.remove_module('Topo')

## Use widgets to modify the parameters

In [ ]:
##### Topography Widgets
TopoModule.show_widgets()

In [ ]:
#### Stop the thread
main.stop()